# 2.4.1 Comprehensions

Efektivní zápis konstrukce listů, dictionary, množin a generátorů.
Umožňuje též filtrování a transformace těchto složených typů.

Syntaxe pro tvorbu listů:

    [ <expression> for x in seq if <condition> ]
    
Pro každý prvek `x` v sekvenci `seq`, který splňuje `<condition>` se vyhodnotí `<expression>` a přidá se do nového pole. Podobně lze tvořit množiny a dictionary. 
[Podrobný popis a příklady.](http://python-3-patterns-idioms-test.readthedocs.io/en/latest/Comprehensions.html)


In [1]:
# List of squares for integers not divisible by 3.

[ i ** 2 for i in range(1, 11) if i%3 > 0]


[1, 4, 16, 25, 49, 64, 100]

In [9]:
l = ( v**2 if v%2==0 else -v**2 for v in range(1,4) )
print(l)  # Not a list just generator.

# we can iterate over it, but only once
for i in l:
    print(i)
    
for i in l:
    print("other round:", i)

<generator object <genexpr> at 0x7f0071e727b0>
-1
4
-9


In [ ]:
# Ekvivalent but using for loop.
l = []
for v in range(1,4):
    if v%2==0:
        l.append(v**2)
    else:
        l.append(-v**2)

In [1]:
# Druhé mocniny lichých čísel. Do dictionary.
print({ i:i**2 for i in range(7) if i%2 == 1 })

{1: 1, 3: 9, 5: 25}


In [2]:
# Druhé mocniny lichých čísel. Do množiny.
print({ i**2 for i in range(7) if i%2 == 1 })

{1, 9, 25}


In [6]:
# Nested  comprehentions are possible.
[
    [i for i in range(n)] 
    for n in range(4)
]

[[], [0], [0, 1], [0, 1, 2]]

In [5]:
d = {1:"A", 2:"B", 3:"C"}

{ k: v.lower()  for k, v in d.items()}

{1: 'a', 2: 'b', 3: 'c'}

**Cvičení**
Přepište následující funkci pomocí comprehension.

In [2]:
def make_sequence( a, b, step=1, power = 1):
    """
    Make a sequence a**p, ..., b**p.
    :param a: float, first value base
    :param b: float, end value base (last base is less then b)
    :param step: float, step of base values
    :param power: float, common power of resulting values
    :return ...
    """
    x = a
    sequence = []
    while x < b:
        sequence.append(x**power)
        x += step
    return sequence

In [45]:
make_sequence(1.1, 4)

[1.1, 2.1, 3.1]

In [47]:
make_sequence(1.1, 10, step = 2)

[1.1, 3.1, 5.1, 7.1, 9.1]

In [3]:
make_sequence(1, 10, step = 2, power=2)

[1, 9, 25, 49, 81]

# 2.4.2 Typová kontrola

- `from typing import *`
- int, float, bool, ...
- `List[<type>]`
- `Dict[<key_type>, <val_type>]`
- `Tuple`
- `Union`
- `set`
- `Any`
- zamýšleno pro statickou kontrolu kódu: ... PyCharm 


In [4]:
from typing import *

def fn(field: List[float]) -> float:
    pass

def fn1(field: Union[List[float], Dict[int, float]]) -> float:
    pass
    
    
class A:
    pass

a: A  = A()

# 2.3.2 Třídy

- `class` definice
- `self` parametr
- dědičnost
- atributy třídy vs. atributy objektu
- `__dict__` - dict obsahující atributy
- `hasattr(obj, attr_name)` 
- knihovna `attrs`



In [3]:
class Point:
    
    @classmethod
    def on_line(cls, t:float, a:'Point', b:'Point') -> 'Point':
        x = a.x + t*b.x
        y = a.y + t*b.y
        return cls(x, y)
    
    def __init__(self, x, y, z=None):
        self.x = x
        self.y = y
        self.z = z
        

aa = Point(1,2)
bb = Point.on_line(0.5, Point(1,2), Point(3,4))
aa.c = 4
print(bb.x, bb.y, hasattr(aa, 'c'))

2.5 4.0 True


In [21]:
class A:
    x:int = 1
    
    def fn(self):
        self.y = self.x

a = A()
b = A()
print(id(a.x), id(b.x))

a.fn()


print(a.__dict__)

9801248 9801248
{'y': 1}


In [1]:
import attrs

a = attrs.field()

print(a.__dict__)


AttributeError: '_CountingAttr' object has no attribute '__dict__'

## Knihovna attrs

Pohodlná tvorba datových tříd.


In [4]:
import attrs

# Object attributes defined by the class attributes.
# Indicated by specifying type.
@attrs.define
class Data:
    p: Point
    i: int = 0
    f: float = 1.0
    s: str = ""
        
    def ifs(self):
        return f"i: {self.i}, f: {self.f}, s: {self.s}"

In [29]:
p = Point(1,2)
d = Data(p)
print(d.ifs())

i: 0, f: 1.0, s: 


In [5]:
# Specification through the `field` is more flexible.
# Class or static methods should be used for non-default construction.

@attrs.define
class Day:
    @classmethod
    def from_int(cls, i:int):
        days = ['Mo', 'Tu', 'We', 'Th', 'Fr', 'Sa', 'Su']
        return cls(days[i]) 
    
    name:str = attrs.field()
    _ii = attrs.field(init=False, default=42)  
    _jj = attrs.ib(type=...) 
    
    

AttributeError: module attrs has no attribute ib

In [37]:
print(Day('Mon'), Day.from_int(0))

Day(name='Mon', _ii=42) Day(name='Mo', _ii=42)


# 2.3.3 Dekorátory

Syntaxe:
```
    @decorator
    def fn(...):
        ...
        
    @other_decorator
    def class(...):
        ...
```

Dekorátor je funkce, která funkci nebo třídu jako parametr a vrací jinou definici.
Lze místo funkce vracet třídu a naopak (speciální využití).



In [44]:
import json

# Concept, of `classmethod` decorator 
#
# def classmethod(fn):
#     def g(self, ...):
#         return fn(self.__class__, ...)
#     return g

class A:
    
    # Decorator change behavior of the function.
    @classmethod
    def create_from_json(cls, fname):
        with open(fname, "r") as f:
            options = json.load(f)
        
        return cls(**options)
    
    def __init__(self, a, b):
        self.a = a
        self.b = b

a = A.create_from_json("decorator_test.json")

print(a.a, a.b)

1 [1, 2, 3]


In [45]:
import time
import math
# Dekorátor je funkce která vyrábí z funkce jinou funkci.

# Změření funkce
def timeit(func):
    def measured(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        t1 = time.time()
        print("Timed ({}): {} s".format(func.__name__, t1 - t0))
        return result
    return measured

@timeit
def iter(x, n):
    for i in range(n):
        x = math.cos(x)
    return x

print(iter(0.2, 10))
print(iter(0.2, 100))
print(iter(0.2, 1000))

Timed (iter): 2.3126602172851562e-05 s
0.7319774252581913
Timed (iter): 2.4080276489257812e-05 s
0.7390851332151607
Timed (iter): 0.0002372264862060547 s
0.7390851332151607


In [49]:
## Dekorátory s parametry

def increase(x):
    def deco_internal(fn):
        def new_fn(a):
            return x + fn(a)
        return new_fn
    return deco_internal

@increase(10)
def square_10(y):
    return y*y

@increase(20)
def square_20(y):
    return y*y

print(square_10(2), square_20(2))


14 24


# 2.3.4 Výjimky


In [16]:
# Raise FileNotFoundError exception.
with open("nofile", "r") as f:
    content = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'nofile'

In [22]:
# Catch FileNotFoundError exception.
try:
    with open("nofile", "r") as f:
        content = f.read()
except FileNotFoundError as e:
    print("Using default config. File not found: ")
    print(e)
    content = "a:None\nb:True"

print(yaml.load(content))

Using default config. File not found: 
[Errno 2] No such file or directory: 'nofile'
a:None b:True


Výjimky jsou v Pythonu používané i místo předchozího testu pomocí podmínky, 
jednak jsou výjimky implementovány efektivně a pro vácevláknové programování nevedou k "race conditions".

Heslo je:
"It's easier to ask forgiveness than it is to get permission." (G. M. Hopper)

In [1]:
import os
# Condition based code.

fname = "nofile"
if os.path.file(fname):
    # If file is removed right now by other thread we get an error.
    with open("nofile", "r") as f:
        content = f.read()
else:
    print("Using default config. File not found: ", fname)
    content = "a:None\nb:True"

print(yaml.load(content))        



AttributeError: module 'posixpath' has no attribute 'file'

In [ ]:
# Další vhodná témata k samostudiu

[Třídy](https://www.tutorialspoint.com/python/python_classes_objects.htm) - 
prakticky nutné pro složitější datové struktury

[Regulární výrazy](https://www.tutorialspoint.com/python/python_reg_expressions.htm) - 
velmi užitečné pro vytahování informací z textových souborů


# 2.3.4 Unit testy

## pytest 
Vyhledává soubory a v nich funkce a třídy pojmenované:`test_*` nebo `*_test`.
Umožňuje jejich spuštění na sadách dat definovaných pomocí dekorátorů.
...

## tox
Provede nainstalování celého balíku pomocí `setup.py` do virtuálního prostředí a teprve tam spouští testy.
Testuje tedy i instalaci a chování po ní.

# Profiling

In [ ]:
# Profiling

# Statistic profiler
import statprof
statprof.start()

# ... some code

statprof.stop()
statprof.display()


# instrumented profiler
import cProfile, pstats, io
pr = cProfile.Profile()
pr.enable()

# ... some code

pr.disable()
s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

# PyCharm

Integrated developlment environment for Python.
Free [PyCharm Community edition](https://www.jetbrains.com/pycharm/download/#section=linux).

- interpreter setup
- browsing through the code
- testing
- debuger


# 2.3.6 Python a SQL

Creating and opearting MySql database: [Tutorial on w3schools](https://www.w3schools.com/python/python_mysql_getstarted.asp) 

Blog about reading SQL using Pandas: 
[SQL for data scientist](https://towardsdatascience.com/sql-in-python-for-beginners-b9a4f9293ecf)
